mindyolo工程结构  └─   │  ├─ 
```python
~path/to/mindyolo
    - /configs    # 配置入口
    - /demo        # 推理  ./predict.py
    - /deploy      # 部署
    - /docs        # 文档、教程
    - /examples    # 使用案例(可当作参考)
    - /mindyolo    # mindyolo 包
    - /requirements
    - /tests       # 用于测试， 数据集可视化、loss测试等
    - /tutorials   # 配置文件的说明(建议看)、云端应用的教程
    - README.md    # 工程介绍，优先看
    - requirements.txt setup.py   # 用于配置/构建项目环境
    - test.py      # 测试
    - train.py     # 训练



```

**环境配置**
```bash
# path/to/mindspore_yolov8
cd mindyolo
pip install -r requirements.txt
pip install -e .   
```

>介绍一下pip install -e . 和 pip install .

`pip install -e .`类似于python setup.py install(传统方法，已经不再推荐，但还是可以用)。
-e 或 --editable：可编辑模式（editable mode），表示以开发模式安装包。
pip install -e . 会在 Python 解释器的 site-packages 目录中创建一个指向当前项目目录的符号链接（symlink），而不会直接复制代码。这样，修改当前目录的源代码后，无需重新安装，修改会立即生效。
pip install -e . 会在 site-packages 目录生成一个 .egg-link 文件，其中记录了当前项目的路径

`pip install .`会根据 setup.py 文件中的配置，将当前目录下的包安装到 Python 环境中。这个过程会将包的文件复制到虚拟环境（或系统环境）中的 site-packages 目录。
pip install . 会在 site-packages 目录中创建一个 .dist-info 文件夹，其中包含包的元数据（如版本、依赖项等）。
与开发模式不同，常规安装会将包的所有文件复制到环境中，而不是创建符号链接。

|命令|作用|安装方式|是否创建符号链接|适用场景|
|---|---|---|---|---|
|pip install .|常规安装|复制到 site-packages|❌ 否|生产环境、正式安装|
|python setup.py install|常规安装|复制到 site-packages|❌ 否|传统方法（已不推荐）|
|pip install -e .|开发模式安装|通过符号链接连接源码	|✅ 是|开发调试|
|python setup.py develop|开发模式安装|通过符号链接连接源码|✅ 是|传统方法（已不推荐）|

- 推荐使用
    - 推荐 pip install . 而不是 python setup.py install，因为 pip 处理依赖项和安装流程更完善。
    - 推荐 pip install -e . 而不是 python setup.py develop，因为 pip 能正确处理依赖项并兼容 pyproject.toml（现代 Python 项目的标准）。


- 如果你在开发一个 Python 包，并希望代码改动后能立即生效，应该使用：
    - pip install -e .


- 如果你只是想安装并使用包，而不进行修改，应该使用：
    - pip install .

数据集格式的转换

```python
# my voc-ObjecDetection-dataset structure
VOC_2007:
~/paddle_yolov8/dataset/VOCDevkit/VOC2007
    - /Annotations   # *.xml
    - /ImageSets
        - /Main      # *.txt(划分的数据集.txt)  txt内每一行都是无后缀的数据文件名
    - /JPEGImages    # *.jpg

# target voc-ObjecDetection-dataset structure
mdspo_voc:
~/path/to/dataset/mdspo_voc
    - /VOCDevkit/VOC2007  # my dataset
        - /Annotations  # *.xml   
        - /ImageSets
            - /Main     # *.txt    
        - /JPEGImages   # *.jpg
    
    - /labels  # line:cls_id normed_x normed_y normed_w normed_h/n
        - /train        # *.txt 
        - /val          # *.txt 
        - /test         # *.txt
    - /images
        - /train        # *.jpg
        - /val          # *.jpg 
        - /test         # *.jpg
    - train.txt  # path/to/*.jpg
    - val.txt    # path/to/*.jpg
    - test.txt   # path/to/*.jpg
```

In [1]:
import os, shutil
import xml.etree.ElementTree as ET
from PIL import Image

# 获取类别的列表  ['head' 'helmet' sfatebelt], 用于获取cls_id
def get_myclasses(clsfile='my_classes.txt'):
    with open(clsfile, "r", encoding='utf-8') as file:
        cls_list = file.readlines()
        return [c.strip() for c in cls_list]

my_classes = get_myclasses()
print(my_classes)  # ['head', 'helmet', 'safetybelt']


['head', 'helmet', 'safetybelt']


In [2]:

def convert_annotations(lbl_filepath, img_data_dir, label_file):
    img_filename = os.path.basename(lbl_filepath).replace('.xml', '.jpg')
    tem_ = Image.open(os.path.join(img_data_dir, img_filename)).convert('RGB')
    w, h = tem_.size
    in_file = open(lbl_filepath, "r", encoding='utf-8')
    tree = ET.parse(in_file)
    root = tree.getroot()

    for obj in root.iter('object'):
        difficult = 0 
        if obj.find('difficult')!=None:
            difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in my_classes or int(difficult)==1:
            continue
        cls_id = my_classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(float(xmlbox.find('xmin').text)), 
             int(float(xmlbox.find('ymin').text)), 
             int(float(xmlbox.find('xmax').text)), 
             int(float(xmlbox.find('ymax').text)))
        x_normed = (b[0]+b[2]) /2 /w
        y_normed = (b[1]+b[3]) /2 /h
        w_normed = abs(b[0]-b[2]) /w
        h_normed = abs(b[1]-b[3]) /h
        # label_file.write(" ".join([str(a) for a in b]) + ' ' + str(cls_id))
        label_file.write(f"{cls_id} {x_normed} {y_normed} {w_normed} {h_normed}\n")
        
    
def copy_files(src_root='dataset/VOCdevkit/VOC2007', 
               tgt_root="mindyolo/dataset/mdspo_voc",
               mode = 'train'):
    os.makedirs(tgt_root, exist_ok=True)
    
    if mode == 'train':
        data_type = 'train'
    elif mode == 'val':
        data_type = 'val'
    else:
        data_type = 'test'
            
    # src
    src_img_dir = os.path.join(src_root, "JPEGImages")
    src_log_path = os.path.join(src_root, f"ImageSets/Main/{data_type}.txt")
    src_lbl_dir = os.path.join(src_root, "Annotations")    
    
    # read data name
    def get_dataNames(logTxt=src_log_path):
        with open(logTxt, "r", encoding='utf-8') as f:
                fnames = f.readlines()
        return fnames
    data_names = get_dataNames()
    
    # # get train.txt  val.txt  test.txt
    # with open(f"{tgt_root}/{data_type}.txt", "w", encoding='utf-8') as fi:
    #     for li in data_names:
    #         li = li.strip()
    #         fi.write(f"../dataset/mdspo_voc/images/{li}\n")
    
    # target img dir
    tgt_img_dir = os.path.join(tgt_root, f"images/{data_type}")
    os.makedirs(tgt_img_dir, exist_ok=True)
    # # copy images
    def copy_imgs(dir1, dir2, fnames):
        for fname in fnames:
            fname = fname.strip()
            src_img_path = os.path.join(dir1, f"{fname}.jpg")
            if os.path.isfile(src_img_path):
                tgt_imag_path = os.path.join(dir2, f"{fname}.jpg")
                shutil.copyfile(src_img_path, tgt_imag_path)
            else:
                print(f"{fname}.jpg file is not file")
                print(f"{src_img_path=}")
                print("------------------------------------------------------")  
    copy_imgs(dir1=src_img_dir, dir2=tgt_img_dir, fnames=data_names)  
    
    # target lbl dir
    tgt_lbl_dir = os.path.join(tgt_root, f"labels/{data_type}")
    os.makedirs(tgt_lbl_dir, exist_ok=True)
    for name_ in data_names:
        xml=name_.strip()
        tgt_lbl_path = os.path.join(tgt_lbl_dir, f"{xml}.txt")
        fil = open(tgt_lbl_path, "w", encoding='utf-8')
        src_xml_path = os.path.join(src_lbl_dir, f"{xml}.xml")
        if os.path.isfile(src_xml_path):
            convert_annotations(src_xml_path, src_img_dir, fil)
        else:
            print(f"{xml}.jpg file is not file")
            print(f"{src_xml_path=}")
            print("------------------------------------------------------")        
        fil.close()
    
copy_files(mode='train')
copy_files(mode='val')   
copy_files(mode='test')

In [7]:
#  数据名得是数字  0001 0002  

def get_num_fname(data_dir="mindyolo/dataset/mdspo_voc", mode="train"):
    if mode == "train":
        data_t = "train"
    elif mode == "val":
        data_t = "val"
    else:
        data_t = "test"
        
    img_d = os.path.join(data_dir, f"images/{data_t}")
    lbl_d = os.path.join(data_dir, f"labels/{data_t}")

    ims = {name.split('.')[0] for name in os.listdir(img_d) if name.endswith('.jpg')}
    lbs = {name.split('.')[0] for name in os.listdir(lbl_d) if name.endswith('.txt')}
    
    # get train.txt  val.txt  test.txt
    fe = open(os.path.join("mindyolo/dataset/mdspo_voc", data_t+".txt"), "w", encoding='utf-8')
    if ims - lbs == set():
        start_num = 1
        for i, na in enumerate(ims, start=start_num):
            fe.write(f"./images/{data_t}/{na}.jpg\n")
            # new_name = f"{i:06d}"
            # fe.write(f"../../dataset/images/{new_name}.jpg\n")
            # src_img = os.path.join(img_d, f"{na}.jpg")        
            # dst_img = os.path.join(img_d, f"{new_name}.jpg")
            # os.rename(src_img, dst_img)
            # src_lbl = os.path.join(lbl_d, f"{na}.txt")
            # dst_lbl = os.path.join(lbl_d, f"{new_name}.txt")
            # os.rename(src_lbl, dst_lbl)
        fe.close()
    else:
        print(f"{ims-lbs=}")
get_num_fname(mode='train')
get_num_fname(mode='val')
get_num_fname(mode='test')

训练配置文件 configs/mdspo_voc.yaml
```yaml

data:
  dataset_name: mdspo_voc

  train_set: dataset/mdspo_voc/train.txt  
  val_set: dataset/mdspo_voc/val.txt  
  test_set: dataset/mdspo_voc/test.txt  
  
  nc: 3

  # class names
  names: [ 'head', 'helmet', 'safetybelt' ]

  train_transforms: []
  test_transforms: []
  ```

主配置文件 configs/yolov8/yolov8s_main.py

```yaml
__BASE__: [
  '../mdspo_voc.yaml',   # 数据集配置
  './hyp.scratch.low.yaml',  # 优化器、损失、数据读取、数据增强low 配置
  './yolov8-base.yaml'  # 模型配置，sync_gn等
]

# weights= ./preweights/yolov8-s_500e_mAP446-3086f0c9.ckpt

overflow_still_update: False
network:
  depth_multiple: 0.33  # scales module repeats
  width_multiple: 0.50  # scales convolution channels
  max_channels: 1024

# 源文件位置
data:
  num_parallel_workers: 8  # 如果报错windwos不支持python多线程就设为1



```

生成dataset/mdspo_voc/annotations/instances_val2017.json文件用于测试，这里是直接调用的pycocotools的api，所以需要我们将数据整理成api要求的格式，直接修改这个脚本就可以mindspore_yolov8\mindyolo\examples\finetune_car_detection\crejson.py 。

In [ ]:
import os
import json
from PIL import Image

# 设置数据集路径
dataset_path = "dataset/mdspo_voc"
images_path = os.path.join(dataset_path, "images/test")
labels_path = os.path.join(dataset_path, "labels/test")

# 类别映射
categories = [
{"id": 0, "name": "head"},
{"id": 1, "name": "helmet"},
{"id": 2, "name": "safetybelt"},
    # 添加更多类别
]


# YOLO格式转COCO格式的函数
def convert_yolo_to_coco(x_center, y_center, width, height, img_width, img_height):
    x_min = (x_center - width / 2) * img_width
    y_min = (y_center - height / 2) * img_height
    width = width * img_width
    height = height * img_height
    return [x_min, y_min, width, height]


# 初始化COCO数据结构
def init_coco_format():
    return {
        "images": [],
        "annotations": [],
        "categories": categories
    }


# 处理每个数据集分区
split=''
coco_format = init_coco_format()
annotation_id = 1
imgs_list = os.listdir(os.path.join(images_path, split))
imgs_list.sort()
for img_name in imgs_list:
    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(images_path, split, img_name)
        label_path = os.path.join(labels_path, split, img_name.replace("jpg", "txt"))

        img = Image.open(img_path)
        img_width, img_height = img.size
        image_info = {
            "file_name": img_name,
            "id": len(coco_format["images"]) + 1,
            "width": img_width,
            "height": img_height
        }
        coco_format["images"].append(image_info)

        if os.path.exists(label_path):
            with open(label_path, "r") as file:
                for line in file:
                    category_id, x_center, y_center, width, height = map(float, line.split())
                    bbox = convert_yolo_to_coco(x_center, y_center, width, height, img_width, img_height)
                    annotation = {
                        "id": annotation_id,
                        "image_id": image_info["id"],
                        "category_id": int(category_id) ,
                        "bbox": bbox,
                        "area": bbox[2] * bbox[3],
                        "iscrowd": 0
                    }
                    coco_format["annotations"].append(annotation)
                    annotation_id += 1

# 为每个分区保存JSON文件
testdir = r"dataset/mdspo_voc/annotations"
os.makedirs(testdir,exist_ok=True)
with open(f"{testdir}/instances_val2017.json", "w") as json_file:
    json.dump(coco_format, json_file, indent=4)


模型配置 configs/yolov8-base 
```yaml

epochs: 500  # total train epochs
per_batch_size: 4
img_size: 640
iou_thres: 0.7
conf_free: True
sync_bn: False    # ddp True
anchor_base: False
opencv_threads_num: 0  # opencv: disable threading optimizations

network:
  model_name: yolov8
  nc: 80  # number of classes
  reg_max: 16

  stride: [8, 16, 32]

  # YOLOv8.0n backbone
  backbone:
    # [from, repeats, module, args]
    - [-1, 1, ConvNormAct, [64, 3, 2]]  # 0-P1/2
    - [-1, 1, ConvNormAct, [128, 3, 2]]  # 1-P2/4
    - [-1, 3, C2f, [128, True]]
    - [-1, 1, ConvNormAct, [256, 3, 2]]  # 3-P3/8
    - [-1, 6, C2f, [256, True]]
    - [-1, 1, ConvNormAct, [512, 3, 2]]  # 5-P4/16
    - [-1, 6, C2f, [512, True]]
    - [-1, 1, ConvNormAct, [1024, 3, 2]]  # 7-P5/32
    - [-1, 3, C2f, [1024, True]]
    - [-1, 1, SPPF, [1024, 5]]  # 9

  # YOLOv8.0n head
  head:
    - [-1, 1, Upsample, [None, 2, 'nearest']]
    - [[-1, 6], 1, Concat, [1]]  # cat backbone P4
    - [-1, 3, C2f, [512]]  # 12

    - [-1, 1, Upsample, [None, 2, 'nearest']]
    - [[-1, 4], 1, Concat, [1] ]  # cat backbone P3
    - [-1, 3, C2f, [256]]  # 15 (P3/8-small)

    - [-1, 1, ConvNormAct, [256, 3, 2]]
    - [[ -1, 12], 1, Concat, [1]]  # cat head P4
    - [-1, 3, C2f, [512]]  # 18 (P4/16-medium)

    - [-1, 1, ConvNormAct, [512, 3, 2]]
    - [[-1, 9], 1, Concat, [1]]  # cat head P5
    - [-1, 3, C2f, [1024]]  # 21 (P5/32-large)

    - [[15, 18, 21], 1, YOLOv8Head, [nc, reg_max, stride]]  # Detect(P3, P4, P5)
```



**train**
```bash
python train.py --config ./configs/yolov8/yolov8s_main.yaml --device_target CPU
```

**infer**
```bash
python demo/predict.py --config ./configs/yolov8/yolov8s_main.yaml --weight /path_to_ckpt/WEIGHT.ckpt --image_path /path/to/img --device_target CPU
```

**test**
```bash
python test.py --config ./configs/yolov8/yolov8s_main.yaml --weight /path_to_ckpt/WEIGHT.ckpt --device_target CPU

```

```yaml

data:
  num_parallel_workers: 8  # 如果报错windwos不支持python多线程就设为1

  # multi-stage data augment
  train_transforms: {
    stage_epochs: [ 490, 10 ],
    trans_list: [
      [
        { func_name: mosaic, prob: 1.0 },
        { func_name: resample_segments },
        { func_name: random_perspective, prob: 1.0, degrees: 0.0, translate: 0.1, scale: 0.5, shear: 0.0 },
        # {func_name: albumentations},
        {func_name: hsv_augment, prob: 1.0, hgain: 0.015, sgain: 0.7, vgain: 0.4},
        {func_name: fliplr, prob: 0.5},
        {func_name: label_norm, xyxy2xywh_: True},
        {func_name: label_pad, padding_size: 160, padding_value: -1},
        {func_name: image_norm, scale: 255.},
        {func_name: image_transpose, bgr2rgb: True, hwc2chw: True}
      ],
      [
        {func_name: letterbox, scaleup: True},
        {func_name: resample_segments},
        {func_name: random_perspective, prob: 1.0, degrees: 0.0, translate: 0.1, scale: 0.5, shear: 0.0},
        # {func_name: albumentations},
        {func_name: hsv_augment, prob: 1.0, hgain: 0.015, sgain: 0.7, vgain: 0.4},
        {func_name: fliplr, prob: 0.5},
        {func_name: label_norm, xyxy2xywh_: True},
        {func_name: label_pad, padding_size: 160, padding_value: -1},
        {func_name: image_norm, scale: 255.},
        {func_name: image_transpose, bgr2rgb: True, hwc2chw: True}
      ]]
  }

  test_transforms: [
    {func_name: letterbox, scaleup: False, only_image: True},
    {func_name: image_norm, scale: 255.},
    {func_name: image_transpose, bgr2rgb: True, hwc2chw: True}
  ]
```